# Example usage

To use `my_package_project` in a project:

In [1]:
import my_package_project

print(my_package_project.__version__)

1.2.0


Package import : 

In [2]:
from datetime import date, timedelta, datetime
from my_package_project.data_treatment import *
from my_package_project.graphs  import *
from my_package_project.operations import *

Range of date and universe

By default, the universe is settled as ['SPY', 'TLT', 'GLD'] i.e. S&P 500 Index, 20+ Year Treasury Bond, Gold (commodities) as it is interesting to see the risk contribution of different asset classes but any list of assets available on yfinance can be set as input.

In [3]:
universe = ['SPY', 'TLT', 'GLD']
initial_date = datetime(2017, 1, 1)
final_date = datetime(2020, 12, 31)
data = get_stocks_data(tickers=universe, start_date=initial_date, end_date=final_date).dropna()

#### Data Treatment :

In [4]:
#Methods from Pybacktestchain :
data_module = DataModule(data=data)
info = Information(s = timedelta(data_module.data.size/len(universe)), data_module=data_module)
t = datetime(2020,12,29)
moments = FirstTwoMoments.compute_information(info, t=t)

Compute the Risk Parity portfolio :

In [5]:
riskparityportfolio =  data_treatment.RiskParity.compute_portfolio_riskparity(info, t=t, information_set=moments)
riskparityportfolio

{'GLD': 0.3359485903774325,
 'SPY': 0.23170624799839645,
 'TLT': 0.432345161624171}

Compute portfolio's volatility : 

In [6]:
vol = data_treatment.portfolio_volatility(riskparityportfolio,information_set=moments)
vol

20.50879603593807

Compute the Risk Parity portfolio with leverage and target volatility : 

In [7]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=vol, leverage_factor=1)

INFO:root:Original Volatility: 20.50879603593807


{'GLD': 0.3359485903774325,
 'SPY': 0.23170624799839645,
 'TLT': 0.432345161624171}

In [8]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=15, leverage_factor=2)

INFO:root:Original Volatility: 20.50879603593807


{'GLD': 0.4914212269536567,
 'SPY': 0.33893688482596235,
 'TLT': 0.6324288771508995}

Compute risk contributions :

In [9]:
data_treatment.compute_risk_contributions(riskparityportfolio, moments)

{'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.333}

#### Graphs : 

In [10]:
visualizer = PortfolioVisualizer(portfolio=riskparityportfolio, information_set=moments)

Plot portfolio initial weight :

In [11]:
initial_weights = visualizer.plot_portfolio_weights()

Plot portfolio risk allocation : 

In [12]:
risk_allocation = visualizer.plot_risk_allocation_pie()

Methods that enable to plot or compute metrics of a portfolio over time are useful in a backtest such as follows:

#### Running a backtest : 

In [13]:
backtest = Backtest_up(
    initial_date = initial_date,
    final_date = final_date,
    information_class = FirstTwoMoments,
    risk_model=StopLoss,
)
backtest.run_backtest()

INFO:root:Running backtest from 2017-01-01 00:00:00 to 2020-12-31 00:00:00.
INFO:root:Retrieving price data for universe
/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2017-01-31 00:00:00
/Users/marc/Desktop/Master 203/M2/S1/Python/Project/pybacktestchain/src/pybacktestchain/broker.py:115: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when 

Risk Contributions: {'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.333}


Portfolio Volatility: 9.64%
Annualized Return: 11.08%
Sharpe Ratio: 0.94


INFO:root: Backtest completed. Final portfolio value: 1609029.7410049438


'backtests/BlackTigerNurse'